In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5033343053488435450
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5726273536
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2723653116953185438
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:2b:00.0, compute capability: 8.6"
]


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50, VGG16, InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder

def cutout(image, label, probability=0.5):
    if np.random.rand() < probability:
        h, w = image.shape[:2]
        size = w // 4
        x1 = np.random.randint(w)
        y1 = np.random.randint(h)
        x2 = np.clip(x1 + size, 0, w)
        y2 = np.clip(y1 + size, 0, h)
        image[y1:y2, x1:x2 :] = 0
    return image, label


df = pd.read_csv("train.csv")
label_encoder = LabelEncoder()
artist_df = df.copy()
artist_train = label_encoder.fit_transform(df['artist'].values)


artist_df['num'] = artist_train
artist_df = artist_df.drop('id', axis=1)
artist_df = artist_df.drop('img_path', axis=1)


artist_df.set_index('num', inplace=True)
artist_df = artist_df.sort_index()


artist_test_dic = artist_df['artist'].to_dict()


# 데이터 노이즈 수정
df.loc[df['id'] == 3896, 'artist'] = 'Titian'
df.loc[df['id'] == 3986, 'artist'] = 'Alfred Sisley'

X_train = df

#X_train, X_val, y_train, y_val = train_test_split(df, df['artist'].values, test_size=0.1)

print("Number of X_train: ", len(X_train))

Number of X_train:  5911


In [2]:
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(artist_train), y=artist_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

display(class_weights_dict)

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(244,244,3), input_tensor=None, pooling=None)

x = GlobalAveragePooling2D()(base_model.output)

predictions = Dense(50, activation='softmax')(x)

x = BatchNormalization()(x)

model = Model(inputs=base_model.input, outputs=predictions)

LearningRate = 1e-3

optimizer = Adam(learning_rate=LearningRate)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['acc'])

CP = ModelCheckpoint(filepath='model/' +
                     'Googlenet-{epoch:03d}-{loss:.4f}-{val_loss:.4f}.hdf5',
                     monitor='val_loss', verbose=1, save_best_only=True, mode='min')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=5, verbose=1, min_lr=0.000001)
CALLBACK = [CP, LR]

early_stopping = EarlyStopping(
    monitor='val_acc',
    min_delta=0.001,
    patience=10,
    verbose=1,
    restore_best_weights=True,
)


DATAGEN_TRAIN = ImageDataGenerator(
    rescale=1/255.0,
    #rotation_range=20,
    width_shift_range=0.25,
    height_shift_range=0.25,
    #shear_range=0.2,
    zoom_range=0.25,
    horizontal_flip=True,
    #vertical_flip=True,
    data_format="channels_last",
    validation_split=0.10
)


TRAIN_GENERATOR = DATAGEN_TRAIN.flow_from_dataframe(
    dataframe=X_train, x_col='img_path', y_col='artist',
    target_size=(244, 244), 
    class_mode='categorical',
    batch_size=32, shuffle=True,
    subset="training"
    #preprocessing_function=cutout
)

VALID_GENERATOR = DATAGEN_TRAIN.flow_from_dataframe(
    dataframe=X_train, x_col='img_path', y_col='artist',
    target_size=(244, 244), 
    class_mode='categorical',
    batch_size=32, shuffle=True,
    subset="validation"
    #preprocessing_function=cutout
)

{0: 0.5373636363636364,
 1: 0.7208536585365853,
 2: 0.8956060606060606,
 3: 1.5975675675675676,
 4: 0.8956060606060606,
 5: 1.8471875,
 6: 3.694375,
 7: 2.003728813559322,
 8: 2.3644,
 9: 1.4595061728395062,
 10: 0.24126530612244898,
 11: 1.906774193548387,
 12: 2.686818181818182,
 13: 1.8187692307692307,
 14: 4.546923076923077,
 15: 0.5795098039215686,
 16: 1.4073809523809524,
 17: 3.9406666666666665,
 18: 1.6419444444444444,
 19: 1.7133333333333334,
 20: 2.814761904761905,
 21: 0.9770247933884297,
 22: 2.2734615384615386,
 23: 1.9380327868852458,
 24: 1.028,
 25: 5.62952380952381,
 26: 1.8471875,
 27: 1.5555263157894736,
 28: 1.299120879120879,
 29: 1.1704950495049504,
 30: 0.6833526011560693,
 31: 3.4770588235294118,
 32: 1.001864406779661,
 33: 0.39016501650165014,
 34: 3.5824242424242425,
 35: 0.5373636363636364,
 36: 0.8325352112676057,
 37: 1.218762886597938,
 38: 0.5073819742489271,
 39: 2.003728813559322,
 40: 1.3908235294117648,
 41: 1.6194520547945206,
 42: 0.653149171270718

Found 5320 validated image filenames belonging to 50 classes.
Found 591 validated image filenames belonging to 50 classes.


In [3]:
history = model.fit(
    TRAIN_GENERATOR,
    epochs=200,
    callbacks=CALLBACK,
    shuffle=True,
    validation_data=VALID_GENERATOR,
    class_weight=class_weights_dict
)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/200
167/167 [==============================] - 66s 353ms/step - loss: 3.6960 - acc: 0.0680 - val_loss: 38.7420 - val_acc: 0.0152

Epoch 00001: val_loss improved from inf to 38.74197, saving model to model\Googlenet-001-3.6960-38.7420.hdf5
Epoch 2/200
167/167 [==============================] - 61s 363ms/step - loss: 3.2103 - acc: 0.1372 - val_loss: 8.1278 - val_acc: 0.0135

Epoch 00002: val_loss improved from 38.74197 to 8.12785, saving model to model\Googlenet-002-3.2103-8.1278.hdf5
Epoch 3/200
167/167 [==============================] - 60s 360ms/step - loss: 2.9078 - acc: 0.1923 - val_loss: 8.3110 - val_acc: 0.0220

Epoch 00003: val_loss did not improve from 8.12785
Epoch 4/200
167/167 [==============================] - 60s 360ms/step - loss: 2.6774 - acc: 0.2376 - val_loss: 3.5524 - val_acc: 0.1726

Epoch 00004: val_loss improved from 8.127

In [4]:
X_test = pd.read_csv("test.csv")

DATAGEN_TEST = ImageDataGenerator(
    rescale=1./255,
    data_format="channels_last"
)

TEST_GENERATOR = DATAGEN_TEST.flow_from_dataframe(
    dataframe=X_test,
    x_col='img_path',
    y_col=None,
    target_size=(244, 244),
    color_mode='rgb',
    class_mode=None,
    batch_size=32,
    shuffle=False
)

model.load_weights('model/Googlenet-023-0.7189-1.6359.hdf5')

TEST_Prediction = model.predict(TEST_GENERATOR, verbose=1)

Found 12670 validated image filenames.
396/396 [==============================] - 132s 332ms/step


In [5]:
artist_num = np.argmax(TEST_Prediction, axis=1)
num_list = pd.DataFrame(artist_num, columns=['artist'])

artist_name = []
for i in num_list['artist']:
    artist_name.append(artist_test_dic[i])

last_ans = pd.read_csv("sample_submission.csv")

last_ans.rename(columns={'artist':'artist_name'}, inplace=True)

last_ans['artist'] = artist_name
last_ans.drop('artist_name', axis=1, inplace=True)

last_ans.to_csv("Googlenet_200_batchnormalization_minlrfix.csv", index=False)